# Volcanic Plume Modeling
## Glaze-Baloga Four-Component Buoyant Plume Model

**Authors:** Prabal Saxena  
**Version:** 1.0  

### Description
This is a code based off an earlier IDL procedure that was created for modeling buoyant rise of a 4-component plume (ash, “dry air”, water vapor, & liquid water) by Glaze and Baloga.  Relevant references describing the code, its' development and application to various solar system bodies are given in the following papers: [Glaze et al. 1996](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/95JD03071) and [Glaze et al. 1997](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/96JD03125).  The original idl code uses a wrapper that calls a file to input the initial volcanic eruption/column parameters assuming a circular vent geometry and can also call routines that provide an atmospheric temperature and pressure profile as well as a water mixing ratio profile for non-dry atmospheres.  We include those files as inline code or functions in this notebook.

The core of the code solves a system of five differential equations describing the thermal dynamics of buoyant eruption columns.  In this case, the plumes are comprised of dry air, magmatic gas, liquid, and solids. The ambient atmosphere may have a different composition to the magmatic gas.  The original program takes as input the initial values for gas content, plume radius, bulk column velocity, and column temperature, and solves five differnetial equations, for plume density, radius, velocity and temperature as functions of z, using the Runge-Kutta method. The IDL code was based on the plumes.pro idl code developed by Glaze et al.

### Citation
If you use this code, please cite:
- Glaze, L. S., & Baloga, S. M. (1996). Sensitivity of buoyant plume heights to ambient atmospheric conditions: Implications for volcanic eruption columns. *Journal of Geophysical Research: Atmospheres*, 101(D1), 1529-1540.
- Glaze, L. S., Baloga, S. M., & Wilson, L. (1997). Transport of atmospheric water vapor by volcanic eruption columns. *Journal of Geophysical Research*, 102(D5), 6099-6108.
- APJ Paper

### Requirements
- numpy >= 1.20
- scipy >= 1.7
- matplotlib >= 3.3

## USER INPUTS
Modify these parameters for your simulation. Atmosphere type and VEI properties are both customizable.

In [ ]:
# =============================================================================
# USER INPUTS - MODIFY THESE FOR YOUR SIMULATION
# =============================================================================

# Path to atmospheric profile file
# Format: altitude(m), pressure(Pa), temperature(K), specific_humidity(kg/kg),
#         liquid_water_ratio, ice_ratio, [optional columns]
ifile = 'ExoCAM/TRAPPIST-1e/aqua/2bar/T1e_2barCO2_aqua_1pa.cam.h0.0100-avg_NW.txt'

# Atmosphere type: 'Earth', 'CO2_Trappist', 'N2_Trappist', 'Earth_Trappist', 'custom'
atmos_type = 'CO2_Trappist'

# VEI case: 'VEI2', 'VEI4', 'VEI5', 'VEI6', 'VEI7', 'VEI8'
vei_case = 'VEI4'

# Output directory
OUTPUT_DIR = 'results'

print(f"Configuration:")
print(f"  Atmospheric profile: {ifile}")
print(f"  Atmosphere type: {atmos_type}")
print(f"  VEI case: {vei_case}")

## Import Required Libraries

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import os
from datetime import datetime

print("Libraries imported successfully")

## Atmospheric Profile Functions
interp_atm is a function to calculate and return an interpolated T, P and water vapor mixing ratio value based upon the atmospheric profile that is provided.

In [ ]:
def interp_atm(z, ifile):
    
    global z_values, pressure, temperature, watervapormixingratio
    
    z_values = []
    pressure = []
    temperature = []
    specifichumidity = []
    watervapormixingratio = []
    waterliquidmixingratio = []
    watericemixingratio = []

    with open(ifile, 'r') as file:
        for line in file:
            line = line.strip()
            
            if not line or line.startswith('#'):
                continue
                
            values = line.split()
            if len(values) >= 7:
                try:
                    z_values.append(float(values[0]))
                    pressure.append(float(values[1]))
                    temperature.append(float(values[2]))
                    specifichumidity.append(float(values[3]))
                    specifichumidityholder = float(values[4])
                    waterliquidmixingratio.append(float(values[4]))
                    waterliquidmixingratioholder = float(values[4])
                    watericemixingratio.append(float(values[5]))
                    watericemixingratioholder = float(values[5])
                    
                    watervapormixingratio.append(
                        (float(values[3]) / (1 - float(values[3]))) + waterliquidmixingratioholder + watericemixingratioholder)
                        
                except ValueError:
                    continue

    interp_pressure = interp1d(z_values, pressure, kind='linear', fill_value='extrapolate')
    interp_temperature = interp1d(z_values, temperature, kind='linear', fill_value='extrapolate')
    interp_watervapor = interp1d(z_values, watervapormixingratio, kind='linear', fill_value='extrapolate')

    p = interp_pressure(z)
    ta = interp_temperature(z)
    w_a = interp_watervapor(z)
    
    return p, ta, w_a


def plotatm(z_values, pressure, temperature, watervapormixingratio):
    """
    Visualize atmospheric profiles.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.invert_yaxis()
    ax2 = ax.twinx()
    ax.plot(temperature, pressure, color="blue", linewidth=2)
    ax.set_xlabel("Temperature [K]")
    ax2.set_ylabel("Altitude [m]")
    ax.set_ylabel("Pressure [Pa]")
    ax2.set_ylim((min(z_values), max(z_values)))
    ax2.plot([], [])
    ax.set_title('Temperature Profile', fontweight="bold", fontsize=14)
    plt.show()
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.invert_yaxis()
    ax2 = ax.twinx()
    ax.plot(watervapormixingratio, pressure, color="red", linewidth=2)
    ax.set_xlabel("Water Vapor Mixing Ratio [kg/kg]")
    ax2.set_ylabel("Altitude [m]")
    ax.set_ylabel("Pressure [Pa]")
    ax2.set_ylim((min(z_values), max(z_values)))
    ax2.plot([], [])
    ax.set_title('Water Vapor Mixing Ratio', fontweight="bold", fontsize=14)
    plt.show()

print("Atmospheric profile functions loaded")

## VEI and Atmosphere Parameter Definitions
Entering initial conditions for the volcanic plume at the vent opening as well as planetary bulk and atmospheere properties. See paper for source from which Trappist planet values and VEI plume parameter values are derived.

In [ ]:
def define(ifile, atmos_type, vei_case='VEI4'):
    """
    Initialize simulation parameters.
    
    Parameters
    ----------
    ifile : str
        Atmospheric profile file path
    atmos_type : str
        Type of atmosphere
    vei_case : str
        Volcanic Explosivity Index case
        
    Returns
    -------
    defs : dict
        Initial altitude (z) and state vector (y)
    """
    global Rd, Rv, Cd, Cv, Cl, Cs, rhol, rhos, MsF, delz, g, alpha
    global atmos, humidity, tau, omega, latht, pressure, temperature, w_a, z
    
    if vei_case == 'VEI2':
        r0 = 30
        u0 = 100
        theta0 = 900
        z0 = 0
        n0 = 0.03
        tau = 1000
    elif vei_case == 'VEI4':
        r0 = 100
        u0 = 250
        theta0 = 900
        z0 = 0
        n0 = 0.03
        tau = 1000
    elif vei_case == 'VEI5':
        r0 = 200
        u0 = 300
        theta0 = 1000
        z0 = 0
        n0 = 0.03
        tau = 1000
    elif vei_case == 'VEI6':
        r0 = 500
        u0 = 400
        theta0 = 1000
        z0 = 0
        n0 = 0.03
        tau = 1000
    elif vei_case == 'VEI7':
        r0 = 1000
        u0 = 600
        theta0 = 1100
        z0 = 0
        n0 = 0.03
        tau = 1000
    elif vei_case == 'VEI8':
        r0 = 2000
        u0 = 700
        theta0 = 1100
        z0 = 0
        n0 = 0.03
        tau = 1000
    else:
        raise ValueError(f"Unknown VEI case: {vei_case}")

    rhol = 1000.
    rhos = 1000.
    alpha = 0.09
    Cv = 2000.
    Cl = 4200.
    Cs = 920.
    Rv = 461.
    latht = 2.257e6
    
    Agol1dg = 6.12
    Agol1eg = 8.01

    if r0 >= 50:
        delz = 5.0
    else:
        delz = r0 / 10

    if atmos_type == 'Earth_Trappist':
        g = Agol1eg
        Cd = 998.
        Rd = 287.
    elif atmos_type == 'CO2_Trappist':
        g = Agol1eg
        Cd = 846.
        Rd = 188.92
    elif atmos_type == 'N2_Trappist':
        g = Agol1eg
        Cd = 1039.
        Rd = 287.
    elif atmos_type == 'Earth':
        g = 9.8
        Cd = 998.
        Rd = 287.
    elif atmos_type == 'custom':
        g = 9.8
        Cd = 998.
        Rd = 287.

    p_interp, ta_interp, w_a_interp = interp_atm(z0, ifile)
    P0 = p_interp
    Ta0 = ta_interp
    z = z0
    w_a = w_a_interp

    rhov0 = P0 / (Rv * float(theta0))
    rhod0 = P0 / (Rd * float(theta0))
    rhoaB0 = (P0 / (Rv * float(Ta0))) * (1 + w_a) / (w_a + Rd / Rv)

    phis0 = ((1 - n0) * rhov0) / ((n0 * rhos) + ((1 - n0) * rhov0))
    phid0 = 0
    phil0 = 0
    phiv0 = 1 - phis0 - phid0 - phil0

    rhoB0 = (rhov0 * phiv0) + (rhod0 * phid0) + (rhol * phil0) + (rhos * phis0)

    MsF = rhos * phis0 * u0 * (r0 ** 2)
    MlF = rhol * phil0 * u0 * (r0 ** 2)
    MvF = rhov0 * phiv0 * u0 * (r0 ** 2)
    MdF = rhod0 * phid0 * u0 * (r0 ** 2)

    m0 = MsF + MvF + MdF + MlF
    CB0 = ((MsF * Cs) + (MvF * Cv) + (MdF * Cd) + (MlF * Cl)) / m0

    nuF = m0 * u0
    TF = m0 * CB0 * float(theta0)

    y = np.zeros(5)
    y[0] = MdF
    y[1] = nuF
    y[2] = TF
    y[3] = MlF
    y[4] = MvF

    omega = cond_rate(z, y)
    defs = {'z': z, 'y': y, 'vei_case': vei_case}
    return defs

print("VEI parameter functions loaded")

## Core Thermodynamic Calculations
This function calculates atmospheric temperature, pressure, and mixing ratio at a given altitude (z) based on different atmospheric models. The ambient air density (rhoaB) is calculated based on temperature, pressure, and mixing ratio. Various parameters related to the volcanic plume and atmospheric conditions are stored in a dictionary (info) and returned.

In [ ]:
def calc(z, y):
    """
    Calculate plume thermodynamic state at altitude z.
    """
    p_interp, ta_interp, w_a_interp = interp_atm(z, ifile)
    Ta = ta_interp
    P = p_interp
    wa = w_a_interp

    rhoadphiad = (P / (Rv * Ta)) / (wa + (Rd / Rv))
    rhoavphiav = rhoadphiad * wa
    rhoaB = rhoavphiav + rhoadphiad
    CaB = (Cd + (wa * Cv)) / (1 + wa)

    MdF = y[0]
    nuF = y[1]
    TF = y[2]
    MlF = y[3]
    MvF = y[4]

    m = MsF + MvF + MdF + MlF
    CB = ((MsF * Cs) + (MvF * Cv) + (MdF * Cd) + (MlF * Cl)) / m
    varm = (P * m * CB) / (Rv * TF)
    rhoB = m * varm / (MvF + (MdF * (Rd / Rv)))

    info = {
        'MdF': MdF, 'MvF': MvF, 'MlF': MlF, 'nuF': nuF, 'TF': TF, 
        'm': m, 'rhoaB': rhoaB, 'varm': varm, 'rhoB': rhoB, 'CB': CB,
        'Ta': Ta, 'P': P, 'rhoadphiad': rhoadphiad, 
        'rhoavphiav': rhoavphiav, 'CaB': CaB
    }
    
    return info

print("Core thermodynamic functions loaded")

## Neutral Buoyancy Height Detection
This block includes a function which prints out values for various plume parameters at a given altitude (z). It calls the calc function to obtain information about the plume and atmospheric conditions.
The printed parameters include plume radius, velocity, temperature, and densities. (Note that this is modified from original code since NBH height was missing crossings due to insufficient threshold. We've switched to a new sign change threshold denoted by the NBH function given here.)

In [ ]:
def find_NBH(z_prev, z_curr, y_prev, y_curr):
    """
    Detect neutral buoyancy height using sign-change detection.
    """
    info_prev = calc(z_prev, y_prev)
    info_curr = calc(z_curr, y_curr)
    
    rho_diff_prev = info_prev['rhoaB'] - info_prev['rhoB']
    rho_diff_curr = info_curr['rhoaB'] - info_curr['rhoB']
    
    if rho_diff_prev * rho_diff_curr < 0:
        alpha_interp = rho_diff_prev / (rho_diff_prev - rho_diff_curr)
        NBH = z_prev + alpha_interp * (z_curr - z_prev)
        rho_plume_NBH = info_prev['rhoB'] + alpha_interp * (info_curr['rhoB'] - info_prev['rhoB'])
        rho_ambient_NBH = info_prev['rhoaB'] + alpha_interp * (info_curr['rhoaB'] - info_prev['rhoaB'])
        return NBH, True, rho_plume_NBH, rho_ambient_NBH
    
    return None, False, None, None


def outpro(z, y, NBH, z_prev=None, y_prev=None, NBH_densities=None):
    """
    Check for NBH crossing during integration.
    """
    if z_prev is not None and y_prev is not None and NBH is None:
        nbh_candidate, found, rho_plume_nbh, rho_ambient_nbh = find_NBH(z_prev, z, y_prev, y)
        if found:
            NBH = nbh_candidate
            NBH_densities = (rho_plume_nbh, rho_ambient_nbh)
            print(f"\n*** Neutral Buoyancy Height detected at: {NBH/1000:.3f} km ***")
    
    return NBH, NBH_densities

print("NBH detection functions loaded")

## Condensation Rate Calculation
Determines the condensation rate (omega) based on the availability of water vapor and the temperature falling below a critical value.

In [ ]:
def cond_rate(z, y):
    """
    Calculate water vapor condensation rate using Clausius-Clapeyron.
    """
    global tau, Rv, latht
    
    info = calc(z, y)
    theta = info["TF"] / (info["m"] * info["CB"])
    MvF = info["MvF"]

    theta_crit = (latht * 373.) / (latht - (Rv * 373. * np.log(info["P"] / 101300.)))

    if (MvF > 0) and (theta < theta_crit):
        omega = 1. / tau if tau != 0 else 0
    else:
        omega = 0

    return omega

print("Condensation rate function loaded")

## Differential Equations (Plume Dynamics)
Solves 5 coupled ODEs for mass, momentum, and energy conservation - calculates the derivatives of various plume variables (MdF, MvF, MlF, nuF, TF) with respect to altitude (z). These derivatives are used in the numerical integration (Runge-Kutta scheme).

In [ ]:
def plumederivs(z, y):
    """
    Calculate derivatives for ODE system.
    Uses global omega value set before integration.
    """
    global Rd, Rv, Cd, Cv, Cl, Cs, rhol, rhos, MsF, delz, g, alpha, omega, latht
    
    dydz = np.zeros(5)
    info = calc(z, y)
    
    c1 = (info['MvF'] + info['MdF'] * (Rd / Rv)) / info['varm']
    
    dMdFdz = 2 * alpha * info['rhoadphiad'] * np.sqrt(info['nuF'] * c1 / info['m'])
    dnuFdz = (g * info['m'] * ((info['rhoaB'] * c1) - info['m'])) / info['nuF']
    dMlFdz = omega * info['MvF'] * info['m'] / info['nuF']  # 
    dMvFdz = (dMdFdz * (info['rhoavphiav'] / info['rhoadphiad'])) - dMlFdz
    dTFdz = ((dMdFdz * info['rhoaB'] * info['CaB'] * info['Ta'] / info['rhoadphiad']) + 
             (latht * dMlFdz) - 
             (info['rhoaB'] * g * (c1 - ((info['MlF'] / rhol) + (MsF / rhos)))))
    
    dydz[0] = dMdFdz
    dydz[1] = dnuFdz
    dydz[2] = dTFdz
    dydz[3] = dMlFdz
    dydz[4] = dMvFdz
    
    return dydz

## Plume Properties and Stopping Criteria

revcalc function is similar to calc but operates on the next altitude level (z+delz). It is used to calculate parameters for the next step in the numerical integration. toptest function determines if the top of the volcanic plume has been reached based on specific conditions involving plume velocity, radius, and their derivatives.

In [ ]:
def revcalc(z, y, yout):
    """
    Calculate plume properties at next step.
    """
    info_out = calc(z + delz, yout)
    info = calc(z, y)

    uout = info_out['nuF'] / info_out['m']
    u = info['nuF'] / info['m']
    rout = np.sqrt((info_out['m']**2) / (info_out['rhoB'] * info_out['nuF']))
    r = np.sqrt((info['m']**2) / (info['rhoB'] * info['nuF']))

    return {'uout': uout, 'u': u, 'rout': rout, 'r': r}


def toptest(z, y, yout, u_cutoff):
    """
    Determine if plume top has been reached.
    """
    result = revcalc(z, y, yout)
    
    if ((result['uout'] <= u_cutoff) or 
        (result['rout'] < 0) or 
        (abs((result['rout'] - result['r']) / result['r']) > 0.4)):
        return 1
    
    return 0

print("Stopping criteria functions loaded")

## Initialize Simulation

In [ ]:
# Extract model identifier
model_id = os.path.splitext(os.path.basename(ifile))[0]

# Initialize parameters
defs = define(ifile, atmos_type, vei_case)
print('Initial values:', defs)

# Plot atmospheric profiles
plotatm(z_values, pressure, temperature, watervapormixingratio)

# Initialize simulation variables
top = 0
u_cutoff = 0.1 if delz <= 1.0 else 10
z = defs['z']
y = defs['y']

NBH = None
NBH_densities = None

# Print initial conditions
print("\nInitial conditions at vent:")
info_initial = calc(z, y)
r_initial = np.sqrt(info_initial['m']**2 / (info_initial['rhoB'] * info_initial['nuF']))
u_initial = info_initial['nuF'] / info_initial['m']
theta_initial = info_initial['TF'] / (info_initial['m'] * info_initial['CB'])
print(f'  Radius: {r_initial:.5f} m')
print(f'  Velocity: {u_initial:.5f} m/s')
print(f'  Temperature: {theta_initial:.5f} K')
print(f'  Plume density: {info_initial["rhoB"]:.5f} kg/m³')
print(f'  Ambient density: {info_initial["rhoaB"]:.5f} kg/m³')
print(f'  Solid mass flux: {MsF:.3e} kg/s')

# Initialize storage arrays
z_profile = []
rho_plume_profile = []
rho_ambient_profile = []
radius_profile = []
velocity_profile = []
temperature_profile = []
z_prev, y_prev = None, None

print("\nSimulation initialized. Ready to run integration.")

## Main Integration Loop

In [ ]:
print("Starting main integration loop...")

test = plumederivs(z, y)
condrates = []

while True:
    info = calc(z, y)
    z_profile.append(z)
    rho_plume_profile.append(info['rhoB'])
    rho_ambient_profile.append(info['rhoaB'])
    radius_profile.append(np.sqrt(info['m']**2 / (info['rhoB'] * info['nuF'])))
    velocity_profile.append(info['nuF'] / info['m'])
    temperature_profile.append(info['TF'] / (info['m'] * info['CB']))
    
    omega = cond_rate(z, y)
    sol = solve_ivp(plumederivs, (z, z + delz), y, method='RK45', dense_output=True)
    yout = sol.sol(z + delz)
    
    top = toptest(z, y, yout, u_cutoff)
    
    z_prev, y_prev = z, y
    z = z + delz
    y = yout
    
    NBH_result, NBH_densities_result = outpro(z, y, NBH, z_prev, y_prev, NBH_densities)
    if NBH_result is not None:
        NBH = NBH_result
    if NBH_densities_result is not None:
        NBH_densities = NBH_densities_result
    
    if top == 1:
        break

print(f'\n*** Plume top reached at: {z/1000:.3f} km ***')
if NBH:
    print(f'Neutral Buoyancy Height: {NBH/1000:.3f} km')
else:
    print('Neutral Buoyancy Height: Not detected')

## Save Results

In [ ]:
z_profile = np.array(z_profile)
z_profile_km = z_profile / 1000
rho_plume_profile = np.array(rho_plume_profile)
rho_ambient_profile = np.array(rho_ambient_profile)
radius_profile = np.array(radius_profile)
velocity_profile = np.array(velocity_profile)
temperature_profile = np.array(temperature_profile)

os.makedirs(OUTPUT_DIR, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"{OUTPUT_DIR}/plume_profile_{vei_case}_{model_id}_{timestamp}.txt"

with open(output_filename, 'w') as f:
    f.write(f"# Volcanic Plume Simulation Results\n")
    f.write(f"# VEI Case: {vei_case}\n")
    f.write(f"# Atmosphere: {atmos_type}\n")
    f.write(f"# Final Height: {z/1000:.3f} km\n")
    if NBH:
        f.write(f"# Neutral Buoyancy Height: {NBH/1000:.3f} km\n")
    else:
        f.write(f"# Neutral Buoyancy Height: Not detected\n")
    f.write(f"#\n")
    f.write(f"# Height(m) PlumeDensity(kg/m³) AmbientDensity(kg/m³) Radius(m) Velocity(m/s) Temperature(K)\n")
    
    for i in range(len(z_profile)):
        f.write(f"{z_profile[i]:.1f} {rho_plume_profile[i]:.6e} {rho_ambient_profile[i]:.6e} ")
        f.write(f"{radius_profile[i]:.2f} {velocity_profile[i]:.2f} {temperature_profile[i]:.2f}\n")

print(f"\nResults saved to: {output_filename}")

## Visualization: Plume Profiles

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0,0].plot(rho_plume_profile, z_profile_km, 'b-', linewidth=2, label='Plume')
axes[0,0].plot(rho_ambient_profile, z_profile_km, 'r--', linewidth=2, label='Ambient')
if NBH:
    axes[0,0].axhline(y=NBH/1000, color='green', linestyle=':', linewidth=2, alpha=0.7, label=f'NBH = {NBH/1000:.1f} km')
axes[0,0].set_xlabel('Density (kg/m³)', fontsize=12)
axes[0,0].set_ylabel('Altitude (km)', fontsize=12)
axes[0,0].set_title('Density Profiles', fontweight='bold')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(radius_profile, z_profile_km, 'purple', linewidth=2)
if NBH:
    axes[0,1].axhline(y=NBH/1000, color='green', linestyle=':', linewidth=1.5, alpha=0.7)
axes[0,1].set_xlabel('Plume Radius (m)', fontsize=12)
axes[0,1].set_ylabel('Altitude (km)', fontsize=12)
axes[0,1].set_title('Plume Width', fontweight='bold')
axes[0,1].grid(True, alpha=0.3)

axes[1,0].plot(velocity_profile, z_profile_km, 'orange', linewidth=2)
if NBH:
    axes[1,0].axhline(y=NBH/1000, color='green', linestyle=':', linewidth=1.5, alpha=0.7)
axes[1,0].set_xlabel('Velocity (m/s)', fontsize=12)
axes[1,0].set_ylabel('Altitude (km)', fontsize=12)
axes[1,0].set_title('Plume Velocity', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(temperature_profile, z_profile_km, 'red', linewidth=2)
if NBH:
    axes[1,1].axhline(y=NBH/1000, color='green', linestyle=':', linewidth=1.5, alpha=0.7)
axes[1,1].set_xlabel('Temperature (K)', fontsize=12)
axes[1,1].set_ylabel('Altitude (km)', fontsize=12)
axes[1,1].set_title('Plume Temperature', fontweight='bold')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle(f'Volcanic Plume Profiles - {vei_case} in {atmos_type}', y=1.00, fontsize=14, fontweight='bold')
plt.savefig(f'{OUTPUT_DIR}/plume_profiles_{vei_case}_{timestamp}.png', dpi=150, bbox_inches='tight')
plt.show()

## Visualization: Buoyancy Profile

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
density_diff = rho_ambient_profile - rho_plume_profile
ax.plot(density_diff, z_profile_km, 'g-', linewidth=3, label='ρ_ambient - ρ_plume')
ax.axvline(x=0, color='black', linestyle='-', linewidth=2, alpha=0.7, label='Neutral Line')
if NBH:
    ax.axhline(y=NBH/1000, color='green', linestyle='--', linewidth=2, label=f'NBH = {NBH/1000:.1f} km')

ax.fill_betweenx(z_profile_km, 0, density_diff, where=(density_diff > 0), alpha=0.3, color='blue', label='Buoyant')
ax.fill_betweenx(z_profile_km, 0, density_diff, where=(density_diff < 0), alpha=0.3, color='red', label='Dense')

ax.set_xlabel('Density Difference (kg/m³)', fontsize=12)
ax.set_ylabel('Altitude (km)', fontsize=12)
ax.set_title('Buoyancy Profile', fontweight='bold', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/buoyancy_profile_{vei_case}_{timestamp}.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n=== SIMULATION COMPLETE ===")